# Projeto 1

João Antônio Misson Milhorim - 11834331

Reynaldo Coronatto Neto - 12547594

## Configurações Iniciais

In [1]:
import math
import glfw
import random
import numpy as np
import OpenGL.GL.shaders
import os

from PIL import Image
from OpenGL.GL import *

In [2]:
class GLWrapper:
    ''' Classe que faz o setup do OpenGL e permite a visualização de imagens '''
    tx, ty, sx, sy = 0.0, 0.0, 1.0, 1.0
    theta = np.radians(0)
        
    def __init__(self, height=700, width=700, title='', figures=None):
        ''' Construtor responsável por chamar funções que configuram o OpenGL '''
        self.show_light = False
        self.figures = figures
        self.set_keys()
        self.window = self.init_window(height, width, title)
        self.add_events(self.window)
        self.program, self.vertex, self.fragment = self.set_shaders()
        self.compile_shaders(self.vertex, 'vertex')
        self.compile_shaders(self.fragment, 'fragment')
        self.attach_shaders(self.program, self.vertex, self.fragment)
        self.link_program(self.program)
    
    def set_keys(self):
        self.KEY_SPACE = 32
        self.KEY_ARROW_DOWN = 264
        self.KEY_ARROW_UP = 265
        self.KEY_ARROW_LEFT = 32
        self.KEY_ARROW_RIGHT = 262
        self.KEY_A = 65
        self.KEY_S = 83
        self.KEY_D = 68
        self.KEY_W = 87
        self.MOUSE_LEFT = 0
        self.MOUSE_RIGHT = 1
    
    def init_window(self, height, width, title):
        ''' Inicializa janela '''
        glfw.init()
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
        window = glfw.create_window(height, width, title, None, None)
        glfw.make_context_current(window)
        
        return window
        
    def key_event(self, window, key, scancode, action, mods):
        ''' Callback para cliques no teclado '''
        if action == 0:
            return

        delta = 0.02
        button = 'Desconhecido'
        
        if key == self.KEY_W: # sobe ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['ty'] += delta
            button = 'W'
        
        if key == self.KEY_S: # desce ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['ty'] -= delta
            button = 'S'
        
        if key == self.KEY_A: # ovni para a esquerda
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['tx'] -= delta
            button = 'A'
        
        if key == self.KEY_D: # ovni para direita
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['tx'] += delta
            button = 'D'
        
        if key == self.KEY_ARROW_RIGHT: # rotaciona sol e lua
            for i in range(len(self.figures['sun'])):
                self.figures['sun'][i]['theta'] -= delta
            for i in range(len(self.figures['moon'])):
                self.figures['moon'][i]['theta'] -= delta
            button = 'Seta direita'
        
        if key == self.KEY_ARROW_UP: # aumenta ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['s'] += delta
            button = 'Seta cima'
        
        if key == self.KEY_ARROW_DOWN: # diminui ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['s'] -= delta
            button = 'Seta baixo'
        
        if key == self.KEY_SPACE: # esconde ou mostra a luz do ovni
            self.show_light = not self.show_light

    def mouse_event(self, window, button, action, mods):
        ''' Callback para cliques no mouse '''
        if action == 0:
            return 
        
        button_str = 'Desconhecido'
        if button == self.MOUSE_LEFT: # carro para a esquerda
            for i in range(len(self.figures['car'])):
                self.figures['car'][i]['tx'] -= 0.02
            button_str = 'Botão Esquerdo'
        
        if button == self.MOUSE_RIGHT: # carro para a direita
            for i in range(len(self.figures['car'])):
                self.figures['car'][i]['tx'] += 0.02
            button_str = 'Botão Direito'
    
    def add_events(self, window):
        ''' Adiciona callbacks dos eventos '''
        glfw.set_key_callback(window, self.key_event)
        glfw.set_mouse_button_callback(window, self.mouse_event)
    
    def set_shaders(self):
        ''' Adiciona shaders '''
        vertex_code = """
                attribute vec3 position;
                attribute vec2 texture_coord;
                varying vec2 out_texture;

                uniform mat4 mat_transform;        

                void main(){
                    gl_Position = mat_transform * vec4(position,1.0);
                    out_texture = vec2(texture_coord);
                }
                """

        fragment_code = """
                uniform vec4 color;
                varying vec2 out_texture;
                uniform sampler2D samplerTexture;

                void main(){
                    vec4 texture = texture2D(samplerTexture, out_texture);
                    gl_FragColor = texture;
                }
        """

        program  = glCreateProgram()
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        return program, vertex, fragment

    def compile_shaders(self, shader, name):
        ''' Compila shaders '''
        glCompileShader(shader)
        if not glGetShaderiv(shader, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(shader).decode()
            print(error)
            raise RuntimeError(f"Erro de compilacao do {name} Shader")
            
    def attach_shaders(self, program, vertex, fragment):
        ''' Associa shaders '''
        glAttachShader(program, vertex)
        glAttachShader(program, fragment)
        
    def link_program(self, program):
        ''' Faz o link do programa '''
        glLinkProgram(program)
        if not glGetProgramiv(program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(program))
            raise RuntimeError('Linking error')

        glUseProgram(program)
    
    def insert_figure_gpu(self, vertices, textures, mat_transform=None):
        ''' Gera um buffer e insere dados vertices e texturas na janela, utiliza matriz
            de transformação, caso seja informada '''
        buffer = glGenBuffers(2)
        
        # vertices
        glBindBuffer(GL_ARRAY_BUFFER, buffer[0])

        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
        glBindBuffer(GL_ARRAY_BUFFER, buffer)

        stride = vertices.strides[0]
        offset = ctypes.c_void_p(0)

        loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(loc)

        glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)
        
        
        # texturas
        glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
        glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
        
        stride = textures.strides[0]
        offset = ctypes.c_void_p(0)
        
        loc_texture_coord = glGetAttribLocation(program, "texture_coord")
        glEnableVertexAttribArray(loc_texture_coord)
        
        glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

    def apply_transformations():
        loc = glGetUniformLocation(self.program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
        
    def get_mat_scale(self, fig_name, xf, yf, pos):
        ''' Gera matriz que realiza alteração da escala da imagem 
            de acordo aos parametros da figura especificada '''
        if pos is not None:
            s = self.figures[fig_name][pos]['s']
        else:
            s = self.figures[fig_name]['s']
            
        mat_scale = np.array([[s, 0.0, 0.0, xf*(1-s)], 
                              [0.0, s, 0.0, yf*(1-s)], 
                              [0.0, 0.0, 0.0, 0.0], 
                              [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_scale

    def get_mat_rotate(self, fig_name, xr, yr, pos):
        ''' Gera matriz que realiza alteração de rotação na imagem 
            de acordo aos parametros da figura especificada'''
        if pos is not None:
            theta = self.figures[fig_name][pos]['theta']
        else:
            theta = self.figures[fig_name]['theta']
            
        mat_rotation = np.array([[np.cos(theta), -np.sin(theta), 0.0, xr-xr*np.cos(theta)+yr*np.sin(theta)], 
                                 [np.sin(theta), np.cos(theta), 0.0, yr-yr*np.cos(theta)-xr*np.sin(theta)], 
                                 [0.0, 0.0, 1.0, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_rotation
    
    def get_mat_translate(self, fig_name, pos):
        ''' Gera matriz que realiza alteração de translação na imagem 
            de acordo aos parametros da figura especificada'''
        if pos is not None:
            tx = self.figures[fig_name][pos]['tx']
            ty = self.figures[fig_name][pos]['ty']
        else:
            tx = self.figures[fig_name]['tx']
            ty = self.figures[fig_name]['ty']
            
        mat_translation = np.array([[1.0, 0.0, 0.0, tx], 
                                    [0.0, 1.0, 0.0, ty], 
                                    [0.0, 0.0, 1.0, 0.0], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_translation
    
    def get_mat_transform(self, fig_name, xr=0, yr=0, xf=0, yf=0, pos=None):
        ''' Gera matriz que realiza a transformação final na imagem 
            combinando as demais matrizes de acordo aos parametros da figura especificada'''
        mat_transform =  self.get_mat_rotate(fig_name, xr, yr, pos) @ \
                         self.get_mat_translate(fig_name, pos) @ \
                         self.get_mat_scale(fig_name, xf, yf, pos)
        return mat_transform

In [ ]:
class Object:
    ''' Classe para lidar com objetdos 3D'''

    def load_model_from_file(filename):
    ''' Loads a Wavefront OBJ file. '''
        objects = {}
        vertices = []
        texture_coords = []
        faces = []
        max_values = []

        material = None

        # abre o arquivo obj para leitura
        for line in open(filename, "r"): ## para cada linha do arquivo .obj
            if line.startswith('#'): continue ## ignora comentarios
            values = line.split() # quebra a linha por espaço
            if not values: continue


            ### recuperando vertices
            if values[0] == 'v':
                vertices.append(values[1:4])


            ### recuperando coordenadas de textura
            elif values[0] == 'vt':
                texture_coords.append(values[1:3])

            ### recuperando faces 
            elif values[0] in ('usemtl', 'usemat'):
                material = values[1]
            elif values[0] == 'f':
                face = []
                face_texture = []
                for v in values[1:]:
                    w = v.split('/')
                    face.append(int(w[0]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)

                faces.append((face, face_texture, material))

        model = {}
        model['vertices'] = vertices
        model['texture'] = texture_coords
        model['faces'] = faces

        return model

    
    # isso precisa entrar em algum lugar!!!!!!
    
#     def load_texture_from_file(texture_id, img_textura):
#         glBindTexture(GL_TEXTURE_2D, texture_id)
#         glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
#         glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
#         glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
#         glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
#         img = Image.open(img_textura)
#         img_width = img.size[0]
#         img_height = img.size[1]
#         image_data = img.tobytes("raw", "RGB", 0, -1)
#         #image_data = np.array(list(img.getdata()), np.uint8)
#         glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

In [3]:
g = GLWrapper(title='Projeto 1')

In [4]:
glfw.show_window(g.window)

In [6]:
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(g.window):
    glfw.poll_events()

    g.apply_transformation()
    
    
    
    glfw.swap_buffers(g.window)

glfw.terminate()